In [16]:
import numpy as np
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import time


class DetectIcon:
    
    def __init__(self, videoPath, iconPathSet):
        # video info
        self.width = 480
        self.height = 270
        self.frameOffset = 0
        self.frameLength = 388800
        self.frameSize = 129600
        self.frameRate = 30
        
        # image processing
        self.kernel_sharpen = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
        self.kernel_blur = np.ones((5,5),np.float32)/25
        self.interplotation = cv2.INTER_CUBIC  ## INTER_NEAREST INTER_LINEAR INTER_AREA INTER_CUBIC INTER_LANCZOS4
        
        # icon info
        self.iconPathSet = iconPathSet
        self.iconSet = [ np.array(Image.open(iconPath)) for iconPath in self.iconPathSet ]
        for i in range(10):
            self.iconSet[0] = cv2.filter2D(self.iconSet[0], -1, self.kernel_blur)
        for i in range(2):
            self.iconSet[1] = cv2.filter2D(self.iconSet[1], -1, self.kernel_blur)
        ## self.iconSet = [ cv2.cvtColor(np.array(Image.open(iconPath)), cv2.COLOR_BGR2GRAY) for iconPath in self.iconPathSet ]

        # Pre-process on brand-icons
        self.iconInfoSet = []
        self.orb = cv2.ORB_create() 
        for icon in self.iconSet:
            kpi, desi = self.orb.detectAndCompute(icon,None)
            self.iconInfoSet.append([kpi, desi])
        
        # Laod the .rgb video file into Memory
        self.videoPath = videoPath
        self.file = open(self.videoPath, 'rb')    ## consider the opne-mode!
        
        # store result
        self.ans = [[], []]
        self.imgsShow = [[], []]
        
        
     
    
    def img_compare(self, iconIndexSet=[0,1]):
        for iconIndex in iconIndexSet:
            for i in range(len(self.imgsShow[iconIndex])):
                cv2.imshow("img_cmp_{}_{}".format(iconIndex, i), self.imgsShow[iconIndex][i])
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
    def output_regularize(self, gap=5, minL=2):
        outPutIndexSet = [[], []]
        for iconIndex in range(2):
            if len(self.ans[iconIndex]) == 1:
                outPutIndexSet[iconIndex].append(0)
                continue
            i = 1
            isCluster = False
            length = 1
            while i < len(self.ans[iconIndex]):
                if self.ans[iconIndex][i][1]-self.ans[iconIndex][i-1][1] < gap*self.frameRate:
                    length += 1
                    if length >= minL:
                        isCluster = True
                elif isCluster:
                    outPutIndexSet[iconIndex].append(i)
                    length = 1
                    isCluster = False
                if i == len(self.ans[iconIndex])-1 and isCluster:
                    outPutIndexSet[iconIndex].append(i)
                i += 1
        
        for iconIndex in range(2):
            self.ans[iconIndex] = [ self.ans[iconIndex][ outPutIndexSet[iconIndex][i] ] for i in range(len(outPutIndexSet[iconIndex])) ]
            self.imgsShow[iconIndex] = [ self.imgsShow[iconIndex][ outPutIndexSet[iconIndex][i] ] for i in range(len(outPutIndexSet[iconIndex])) ]

            
    def cut_copy_image(self, img, size, corner=[0,0]):
        # initialization
        width = 480
        height = 270
        cWidth = int(size[0])
        cHeight = int(size[1])

        frameSize = width*height
        framOffsetWidth = corner[0]
        framOffsetHeight = corner[1]
        lineSkip = width-cWidth


        cut = np.full((cHeight,cWidth,3), 0, np.uint8)

        # cut-copy image
        ind = framOffsetWidth + framOffsetHeight*width
        for y in range(cHeight):
            for x in range(cWidth):
                cut[y][x] = [img[ind], img[ind + frameSize], img[ind + frameSize*2]]
                ind += 1
            ind += lineSkip
        return cut
    
    
    def detect_one_frame(self, frameNumber, iconIndex, threshHold=10, passLine=0.75, imshow=True, corner=[0,0], zoomProp=1/6):
        isFound = False
        
        # initialization
        pWidth = int(self.width * (1-2*zoomProp))
        pHeight = int(self.height * (1-2*zoomProp))
        frameArr = np.full((pHeight, pWidth, 3), 0, np.uint8)
        
        # locate & laod .rgb frame
        position = self.frameLength * frameNumber
        self.file.seek(position, 0)
        frameByte = self.file.read(self.frameLength)

        # transfer .rgb frame to ndarray
        frameArr = self.cut_copy_image(frameByte, [pWidth, pHeight], corner)
        

        # get key-point info of Detecting-Area
        alignFrameArr = cv2.resize(frameArr, (480, 270), interpolation=self.interplotation)
        ## alignFrameArr = cv2.filter2D(alignFrameArr, -1, kernel=self.kernel)
        
        kpf, desf = self.orb.detectAndCompute(alignFrameArr,None)

        # compare with brand-icon info
        bf = cv2.BFMatcher()
        matches = bf.knnMatch(self.iconInfoSet[iconIndex][1], desf, k=2)
        
        # collect good matches
        good = []
        imgMatchNumber = []
        for m,n in matches:
            if m.distance < passLine*n.distance:
                good.append([m])
                if kpf[m.trainIdx].pt not in imgMatchNumber:
                    imgMatchNumber.append(kpf[m.trainIdx].pt)
        
        
        # Continue zoom-in 
        if len(good) >= threshHold:
            isFound = True
            if imshow:
                ## print("  |--> Zoom-In Detection:")
                matchCompImg = cv2.drawMatchesKnn(self.iconSet[iconIndex], self.iconInfoSet[iconIndex][0], alignFrameArr, kpf, good, None, flags=2)  ## what is 'flags'?
                plt.figure("matchCompImg"); plt.imshow(matchCompImg); plt.axis('off'); plt.show()
        
        # return result
        ## if isFound:
           ## print("        | Pass!, frame={}, icon_id={}, #good={} |".format(frameNumber, iconIndex, len(good)))
        ## else:
           ## print("        | Not Pass!, frame={}, icon_id={}, #good={} |".format(frameNumber, iconIndex, len(good)))
        ## return isFound
    
    
    def detect_zoom_in(self, img, iconIndex, threshHold=10, passLine=0.75, imshow=True, corner=[0,0], zoomProp=1/6):
        isFound = False
        
        # initialization
        pWidth = int(self.width * (1-2*zoomProp))
        pHeight = int(self.height * (1-2*zoomProp))

        # regulize corner
        corner[0] = min( int(corner[0]), int(self.width*2*zoomProp) ); corner[0] = max(corner[0], 0)
        corner[1] = min( int(corner[1]), int(self.height*2*zoomProp) ); corner[1] = max(corner[1], 1)
        
        img = img[ corner[1]:corner[1]+pHeight, corner[0]:corner[0]+pWidth]

        # get key-point info of Detecting-Area
        img = cv2.resize(img, (480, 270), interpolation=self.interplotation)
        ## img = cv2.filter2D(img, -1, kernel=self.kernel)
        kpf, desf = self.orb.detectAndCompute(img,None)
        if len(kpf)<1 or len(desf)<1:
            return [False, [-1, -1], img, None, None]
        

        # compare with brand-icon info
        bf = cv2.BFMatcher()
        matches = bf.knnMatch(self.iconInfoSet[iconIndex][1], desf, k=2)
        
        # collect good matches
        good = []
        imgMatchNumber = []
        center = np.array([0, 0])
        for i, pair in enumerate(matches):
            try:
                m, n = pair
                if m.distance < passLine*n.distance:
                    good.append([m])
                    if kpf[m.trainIdx].pt not in imgMatchNumber:
                        imgMatchNumber.append(kpf[m.trainIdx].pt)
                    center = np.array([center[0]+kpf[m.trainIdx].pt[0], center[1]+kpf[m.trainIdx].pt[1]])
            except ValueError:
                pass
        if len(good):
            center = center / len(good)
        
        # Continue zoom-in 
        if len(good) >= threshHold and len(imgMatchNumber) > 0.6*len(good): isFound = True

        # return result
        if isFound:
            ## print("        | Pass!, icon_id={}, #good={}, len(imgMatchNumber)={} |".format(iconIndex, len(good), len(imgMatchNumber)) )
            if imshow:
                print("    |--> Zoom-In Detection: #good =", len(good))
                matchCompImg = cv2.drawMatchesKnn(self.iconSet[iconIndex], self.iconInfoSet[iconIndex][0], img, kpf, good, None, flags=2)  ## what is 'flags'?
                plt.figure("matchCompImg"); plt.imshow(matchCompImg); plt.axis('off'); plt.show()
        ## else:
            ## print("        | Not Pass!, icon_id={}, #good={}, len(imgMatchNumber) |".format(iconIndex, len(good), len(imgMatchNumber)) )
        return [isFound, center, img, kpf, good]
    
    
    def detect_video_segment(self, iconIndexSet=[0,1], threshHold=[10,8,8,8,8], passLine=0.75, start=0, end=10, imshow=True, corner=[0,0], zoomProp=1/6, step=1):
        begin = time.clock()
        
        # Initialization
        ans = []
        imgsShow = []
        frameOffset = start*30
        pWidth = int(self.width * (1-2*zoomProp))
        pHeight = int(self.height * (1-2*zoomProp))
        frameArr = np.full((180,320,3), 0, np.uint8)
        position = self.frameLength * frameOffset

        # Detect brand-icons in video
        print("Detecting...")
        while frameOffset <= end*30:
            if frameOffset%90 == 0: print("    {}s video detected.".format( (frameOffset-start*30)/30 ) )
            
            # locate & laod .rgb frame
            position = frameOffset * self.frameLength
            self.file.seek(position, 0)
            frameByte = self.file.read(self.frameLength)

            # transfer .rgb frame
            frameArr = self.cut_copy_image(frameByte, [pWidth, pHeight], corner)

            # get key-point info of current-frame
            alignFrameArr = cv2.resize(frameArr, (480, 270), interpolation=self.interplotation)
            ## alignFrameArr = cv2.filter2D(alignFrameArr, -1, kernel=self.kernel)
            kpf, desf = self.orb.detectAndCompute(alignFrameArr,None)
            if len(kpf)<1 or len(desf)<1:
                frameOffset += step
                continue

            # compare with brand-icon info
            bf = cv2.BFMatcher()
            for iconIndex in iconIndexSet:
                iconInfo = self.iconInfoSet[iconIndex]
                matches = bf.knnMatch(iconInfo[1],desf, k=2)  ## what is 'k'?
                # collect good matches & calculate gravity-center
                good = []
                center = np.array([0, 0])
                for m,n in matches:
                    if m.distance < passLine*n.distance:
                        good.append([m])
                        center = np.array([center[0]+kpf[m.trainIdx].pt[0], center[1]+kpf[m.trainIdx].pt[1]])
                if len(good):
                    center = center / len(good)
                
                ## Continue Zoom-In:
                prop = 1/5
                zoomCorner = [ int(center[0]-0.5*self.width*(1-2*prop)), int(center[1]-0.5*self.height*(1-2*prop)) ]
                if len(good) >= threshHold[0]:
                    ## print("      B")
                    isFound, center, img1, kpf1, good1 = self.detect_zoom_in(alignFrameArr, iconIndex, threshHold=threshHold[1], corner=zoomCorner, passLine=passLine, zoomProp=1/5, imshow=imshow)
                    if isFound:
                        ## print("      C"); 
                        zoomCorner = [ int(center[0]-0.5*self.width*(1-2*prop)), int(center[1]-0.5*self.height*(1-2*prop)) ]
                        isFound, center, img2, kpf2, good2 = self.detect_zoom_in(img1, iconIndex, threshHold=threshHold[2], corner=zoomCorner, passLine=passLine, zoomProp=1/5, imshow=imshow)
                    if isFound:
                        ## print("      D"); 
                        zoomCorner = [ int(center[0]-0.5*self.width*(1-2*prop)), int(center[1]-0.5*self.height*(1-2*prop)) ]
                        isFound, center, img3, kpf3, good3 = self.detect_zoom_in(img2, iconIndex, threshHold=threshHold[3], corner=zoomCorner, passLine=passLine, zoomProp=1/5, imshow=imshow)
                    if isFound:
                        ## print("      E"); 
                        zoomCorner = [ int(center[0]-0.5*self.width*(1-2*prop)), int(center[1]-0.5*self.height*(1-2*prop)) ]
                        isFound, center, img4, kpf4, good4 = self.detect_zoom_in(img3, iconIndex, threshHold=threshHold[4], corner=zoomCorner, passLine=passLine, zoomProp=1/5, imshow=imshow)
                    if isFound:
                        matchCompImg = cv2.drawMatchesKnn(self.iconSet[iconIndex], iconInfo[0], img4, kpf4, good4, None, flags=2)
                        imgShow = np.hstack((alignFrameArr, matchCompImg))
                        self.imgsShow[iconIndex].append(imgShow)
                        self.ans[iconIndex].append( [iconIndex, frameOffset] )
                        if imshow:
                            ## print("Alternative-Frame: \nframeOffset =", frameOffset, "time =", int(frameOffset/30), "  icon_ind =", iconIndex, "  #good =", len(good))
                            matchCompImg = cv2.drawMatchesKnn(self.iconSet[iconIndex], iconInfo[0], alignFrameArr, kpf, good, None, flags=2)  ## what is 'flags'?
                            ## print("      A"); 
                            plt.figure("matchCompImg"); plt.imshow(matchCompImg); plt.axis('off'); plt.show()
                            
            frameOffset += step

        # Return
        print("Detection finished in {} seconds.".format(time.clock()-begin))
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        return [self.ans, self.imgsShow]

    
    

    
    

detect_3 = DetectIcon(videoPath=r'./data/data_test3.rgb', iconPathSet=[r'./data/ae_logo.jpg', r'./data/hrc_logo.jpg'])  ## r'./data/hrc_logo.jpg'
## detect_3.detect_one_frame(2400, 0, threshHold=1, corner=[0,0], zoomProp=1/6)
detect_3.detect_video_segment(iconIndexSet=[0], threshHold=[2,1,2,5,5], passLine=0.75, start=65, end=67, step=3, zoomProp=1/6, imshow=False)
detect_3.detect_video_segment(iconIndexSet=[1], threshHold=[2,2,2,3,3], passLine=0.65, start=234, end=235, step=1, zoomProp=1/6, imshow=False)
print("len_ans", len(detect_3.ans[0]), len(detect_3.ans[1])); print("ans", detect_3.ans)
detect_3.output_regularize()
print("len_ans", len(detect_3.ans[0]), len(detect_3.ans[1])); print("ans", detect_3.ans)
detect_3.img_compare()


Detecting...
    1.0s video detected.
Detection finished in 2.6578454992263687 seconds.
Detecting...
    0.0s video detected.
Detection finished in 4.264242780973291 seconds.
len_ans 2 1
ans [[[0, 1983], [0, 1995]], [[1, 7049]]]
len_ans 1 1
ans [[[0, 1995]], [[1, 7049]]]
